# How to cite 
@software{Alireza_Soroudi_PyomoOptimization, author = {Alireza Soroudi, Alireza}, license = {MIT}, title = {{PyomoOptimization}}, url = {https://github.com/OptimizationExpert/Pyomo%7D }

In [1]:
from pyomo.environ import *
import numpy as np 
import matplotlib.pyplot as plt

# Bi-level optimization problem
$$\min_{0 \leq x} x-4y \ \ \ \ \ \ $$
$$\ \ \ \ Subject \ to$$
$$ \ \ \ \min_{y\geq 0} y $$
$$ -x-y +3\leq 0 \ : \mu_1 $$
$$ -2x+y\leq 0 \ : \mu_2  $$
$$ 2x+y -12\leq 0 \ : \mu_3  $$
$$ -3x+2y+4\leq 0 \ : \mu_4  $$

## The min problem should be transformed to max form

$$\max_{x,y} f(x,y)$$
$$ h_i(x,y) =0 :  \lambda_i$$
$$ g_j(x,y) \leq 0 : \mu_j $$


# Bi-level optimization problem
$$\min_{0 \leq x} x-4y \ \ \ \ \ \ $$
$$\ \ \ \ Subject \ to$$
$$ \ \ \ \max_{y} -y $$
$$ -x-y +3\leq 0 \ : \mu_1 $$
$$ -2x+y\leq 0 \ : \mu_2  $$
$$ 2x+y -12\leq 0 \ : \mu_3  $$
$$ -3x+2y+4\leq 0 \ : \mu_4  $$
$$ -y\leq 0 \ : \mu_5  $$

We need to replace the lower problem with its KKT conditions 

# KKT conditions: 
- Create Lagrangian      
$$\mathcal{L} = f(x,y) + \sum_i \lambda_i h_i(x,y) + \sum_j \mu_j g_j(x,y) $$

$$ \mathcal{L} = -y +  (-x-y +3)\mu_1 
+ (-2x+y)\mu_2 
+ (2x+y -12)\mu_3 
+ (-3x+2y+4)\mu_4  
+ (-y)\mu_5  $$

- Differentiate to the lower leb=vel variable (y) = 0       
$$ \frac{\partial \mathcal{L}}{\partial y} = -1 -\mu_1 + \mu_2 +\mu_3 + 2\mu_4 -\mu_5 =0  $$

- Complementarity slackness conditions 
$$  (-x-y +3)\mu_1 = 0 $$
$$ (-2x+y)\mu_2 = 0   $$
$$ (2x+y -12)\mu_3 = 0 $$
$$ (-3x+2y+4)\mu_4 = 0 $$
$$ (-y)\mu_5 = 0  $$
$$ \mu_i \geq 0 $$

- Original conditions: 
$$ -x-y +3\leq 0 \ : \mu_1 $$
$$ -2x+y\leq 0 \ : \mu_2  $$
$$ 2x+y -12\leq 0 \ : \mu_3  $$
$$ -3x+2y+4\leq 0 \ : \mu_4  $$
$$ -y\leq 0 \ : \mu_5  $$

# Pyomo code

In [2]:
# create a model
model = AbstractModel()

# declare decision variables
model.x = Var(initialize = 4,within=NonNegativeReals)
model.y = Var(initialize = 4,within=NonNegativeReals)
model.mu1 = Var(initialize = 0, within=NonNegativeReals)
model.mu2 = Var(initialize = 0, within=NonNegativeReals)
model.mu3 = Var(initialize = 0, within=NonNegativeReals)
model.mu4 = Var(initialize = 0, within=NonNegativeReals)
model.mu5 = Var(initialize = 0, within=NonNegativeReals)

# declare constraints
model.C1 = Constraint(expr = (-model.x-model.y+3) <= 0)
model.C2 = Constraint(expr = (-2*model.x+model.y) <= 0)
model.C3 = Constraint(expr = (2*model.x+model.y-12) <= 0)
model.C4 = Constraint(expr = (-3*model.x+2*model.y+4) <= 0)
model.C5 = Constraint(expr = (-model.y) <= 0)

model.lagrange = Constraint(expr = -1-model.mu1+model.mu2+model.mu3+2*model.mu4-model.mu5 == 0)
model.Com1 = Constraint(expr = (-model.x-model.y+3)*model.mu1 == 0)
model.Com2 = Constraint(expr = (-2*model.x+model.y)*model.mu2 == 0)
model.Com3 = Constraint(expr = (2*model.x+model.y-12)*model.mu3 == 0)
model.Com4 = Constraint(expr = (-3*model.x+2*model.y+4)*model.mu4 == 0)
model.Com5 = Constraint(expr = (-model.y)*model.mu5 == 0)


# declare objective
model.obj = Objective(expr = model.x -4*model.y, sense=minimize)
opt = SolverFactory('ipopt')
instance = model.create_instance()# solves and updates instance
results = opt.solve(instance)
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)

print('OF = ', round(value(instance.obj),4) )
print('x = ', round(value(instance.x),4) )
print('y = ', round(value(instance.y),4) )

print('mu_1 = ', round(value(instance.mu1),4) )
print('mu_2 = ', round(value(instance.mu2),4) )
print('mu_3 = ', round(value(instance.mu3),4) )
print('mu_4 = ', round(value(instance.mu4),4) )
print('mu_5 = ', round(value(instance.mu5),4) )

feasible
OF =  -12.0
x =  4.0
y =  4.0
mu_1 =  0.0
mu_2 =  -0.0
mu_3 =  0.5
mu_4 =  0.25
mu_5 =  0.0
